In [1]:
#importing the relevant libraries
import pandas as pd
import numpy as np
import os
from glob import glob
import sys
pd.set_option('display.max_columns', None, 'display.max_rows', None)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
data_wheather = pd.read_csv('./cleaned/wheather.CSV',decimal=",")
data_wheather['DATE_ONLY'] = pd.to_datetime(data_wheather['TIME_UTC_STR']).dt.strftime('%m/%d/%Y')
data_wheather = data_wheather.sort_values(by=['DATE_ONLY']).drop(columns=["Unnamed: 8"])

data_wheather = data_wheather.assign(season_year = lambda x: x['file_loc'].str.split('\\').str[1])\
                .assign(location = lambda x: x['file_loc'].str.split('\\').str[2])\
                .assign(match_type = lambda x: x['file_loc'].str.split('\\').str[3])\
                .assign(match_name = lambda x: x['file_loc'].str.split('\\').str[4])\
                .assign(match_name = lambda x: x['match_name'].str.split('_').str[2])\
                .assign(match_name = lambda x: x['match_name'].str.split('.').str[0])\
                .drop(columns = ['file_loc'])

In [3]:
aggregation = {'AIR_TEMP':[('mean_air_temp','mean'),
                ('std_air_temp','std'),
                ('med_air_temp','median'),
                ('max_air_temp','max'),
                ('min_air_temp','min'),
                ('range_air_temp',lambda x: max(x) - min(x)),
                ('skew_air_temp','skew')],
              'TRACK_TEMP' : [('mean_track_temp','mean'),
                ('std_track_temp','std'),
                ('med_track_temp','median'),
                ('max_track_temp','max'),
                ('min_track_temp','min'),
                ('range_track_temp',lambda x: max(x) - min(x)),
                ('skew_track_temp','skew')],
                'HUMIDITY' : [('mean_HUMIDITY','mean'),
                            ('std_HUMIDITY','std'),
                            ('med_HUMIDITY','median'),
                            ('max_HUMIDITY','max'),
                            ('min_HUMIDITY','min'),
                            ('range_HUMIDITY',lambda x: max(x) - min(x)),
                            ('skew_HUMIDITY','skew')],
                'PRESSURE' : [('mean_PRESSURE','mean'),
                    ('std_PRESSURE','std'),
                    ('med_PRESSURE','median'),
                    ('max_PRESSURE','max'),
                    ('min_PRESSURE','min'),
                    ('range_PRESSURE',lambda x: max(x) - min(x)),
                    ('skew_PRESSURE','skew')],

            'WIND_SPEED' : [('mean_WIND_SPEED','mean'),
                          ('std_WIND_SPEED','std'),
                          ('med_WIND_SPEED','median'),
                          ('max_WIND_SPEED','max'),
                          ('min_WIND_SPEED','min'),
                          ('range_WIND_SPEED',lambda x: max(x) - min(x)),
                          ('skew_WIND_SPEED','skew'),
                        ]
              }

#data_wheather[aggregation.keys] = data_wheather[aggregation.keys()].apply(pd.to_numeric)

data_wheather['match_key'] = data_wheather['DATE_ONLY'] + data_wheather['match_name']
df = data_wheather.groupby(['match_key','DATE_ONLY','location','season_year','match_name','match_type'], as_index=True).agg(aggregation).reset_index()
df.columns = df.columns.droplevel()
column_names = df.columns.values
column_names[:6] = ['match_key','DATE_ONLY','location','season_year','match_name','match_type']
df.columns = column_names
df = df.drop(columns = ['match_key'])
#df.to_csv('./cleaned/weather_processed.csv',index = False)

In [8]:
df.shape

(175, 40)

In [4]:
df.head()

,DATE_ONLY,location,season_year,match_name,match_type,mean_air_temp,std_air_temp,med_air_temp,max_air_temp,min_air_temp,range_air_temp,skew_air_temp,mean_track_temp,std_track_temp,med_track_temp,max_track_temp,min_track_temp,range_track_temp,skew_track_temp,mean_HUMIDITY,std_HUMIDITY,med_HUMIDITY,max_HUMIDITY,min_HUMIDITY,range_HUMIDITY,skew_HUMIDITY,mean_PRESSURE,std_PRESSURE,med_PRESSURE,max_PRESSURE,min_PRESSURE,range_PRESSURE,skew_PRESSURE,mean_WIND_SPEED,std_WIND_SPEED,med_WIND_SPEED,max_WIND_SPEED,min_WIND_SPEED,range_WIND_SPEED,skew_WIND_SPEED
0,01/04/2017,Mexico,2016-2017,Groups Qualifying Session,Q,27.101836,0.574968,27.4444,27.7222,26.3889,1.3333,-0.406788,21.574100,0.209570,21.6667,21.6667,21.1111,0.5556,-1.855802,31.142857,0.354169,31.0,32,31,1,2.117634,779.436476,0.198200,779.4985,779.719,779.143,0.576,-0.184513,7.165413,1.516478,8.04672,8.04672,3.21869,4.82803,-1.417020
1,01/04/2017,Mexico,2016-2017,Non Qualifying Practice 1,FP1,16.888880,1.024915,17.3333,19.2222,15.0556,4.1666,-0.614885,21.913584,0.816790,21.6667,23.3333,20.5556,2.7777,0.158910,55.155556,0.366529,55.0,56,55,1,1.966912,780.824667,0.061080,780.8020,780.938,780.734,0.204,0.158626,2.610713,1.690992,3.21869,4.82803,0.00000,4.82803,-0.154161
2,01/04/2017,Mexico,2016-2017,Non Qualifying Practice 2,FP2,25.546313,0.219368,25.6667,25.7778,25.1667,0.6111,-0.918498,20.555600,0.000000,20.5556,20.5556,20.5556,0.0000,0.000000,33.266667,0.449776,33.0,34,33,1,1.111663,780.391367,0.062756,780.3960,780.531,780.260,0.271,0.475190,5.954573,2.902056,6.43738,14.48410,1.60934,12.87476,0.898993
3,01/04/2017,Mexico,2016-2017,Race,Race,27.262177,0.175923,27.2778,27.5000,26.9444,0.5556,-0.645954,23.245589,0.204359,23.3333,23.3333,22.7778,0.5555,-1.927487,34.368421,0.486664,34.0,35,34,1,0.560401,777.901351,0.120517,777.8900,778.229,777.687,0.542,0.643746,7.482039,3.310649,6.43738,19.31210,3.21869,16.09341,1.271079
4,01/04/2017,Mexico,2016-2017,Super Pole Session,Q,27.817464,0.145223,27.8889,27.9444,27.5556,0.3888,-1.383638,22.222200,0.000000,22.2222,22.2222,22.2222,0.0000,0.000000,30.642857,0.497245,31.0,31,30,1,-0.670360,778.876857,0.060905,778.8890,778.974,778.770,0.204,-0.435894,6.782234,3.167426,6.43738,16.09340,3.21869,12.87471,1.983345


In [6]:
#drop seasong year
#make changes to DATE_ONLY to extract year 

str

In [10]:
dr = pd.read_csv('./cleaned/race_final_processed.csv')
final_data = dr.merge(df, how = 'left', on = ['location','season_year','match_name','match_type'])
final_data_with_weather = final_data[final_data['mean_air_temp'].notnull()]
final_data_with_weather.shape

(664, 74)

In [11]:
final_data_with_weather.head()

,DRIVER_NAME,location,season_year,match_name,match_type,GROUP,TEAM,mean_KPH,std_KPH,med_KPH,max_KPH,min_KPH,range_KPH,skew_KPH,mean_S1,std_S1,med_S1,max_S1,min_S1,range_S1,skew_S1,mean_S2,std_S2,med_S2,max_S2,min_S2,range_S2,skew_S2,mean_S3,std_S3,med_S3,max_S3,min_S3,range_S3,skew_S3,Total_Lap_Num,PIT_TIME_cnt,cnt_CROSSING_FINISH_LINE_IN_PIT,DATE_ONLY,mean_air_temp,std_air_temp,med_air_temp,max_air_temp,min_air_temp,range_air_temp,skew_air_temp,mean_track_temp,std_track_temp,med_track_temp,max_track_temp,min_track_temp,range_track_temp,skew_track_temp,mean_HUMIDITY,std_HUMIDITY,med_HUMIDITY,max_HUMIDITY,min_HUMIDITY,range_HUMIDITY,skew_HUMIDITY,mean_PRESSURE,std_PRESSURE,med_PRESSURE,max_PRESSURE,min_PRESSURE,range_PRESSURE,skew_PRESSURE,mean_WIND_SPEED,std_WIND_SPEED,med_WIND_SPEED,max_WIND_SPEED,min_WIND_SPEED,range_WIND_SPEED,skew_WIND_SPEED
0,MA Qing Hua,Buenos,2016-2017,Groups Qualifying Session,Q,4.0,TECHEETAH,55.950000,74.034080,55.95,108.3,3.6,104.7,0.000000,1.205626e+06,1.673677e+06,1205626.5,2389095.0,22158.0,2366937.0,0.000000,28569.500000,2348.301620,28569.5,30230.0,26909.0,3321.0,0.000000,33244.500000,132.228968,33244.5,33338.0,33151.0,187.0,0.000000,2.0,2,0,02/18/2017,26.299227,0.067998,26.2778,26.4444,26.2222,0.2222,0.634238,26.250000,0.243363,26.1111,26.6667,26.1111,0.5556,1.195861,74.931818,0.899554,75.0,76.0,73.0,3.0,-0.665006,1008.236136,0.126201,1008.230,1008.470,1008.030,0.440,0.053279,4.462272,2.049350,4.828030,8.04672,0.00000,8.04672,0.095071
1,MA Qing Hua,HongKong,2016-2017,Groups Qualifying Session,Q,2.0,TECHEETAH,60.720000,44.706677,83.70,103.4,9.6,93.8,-0.504993,1.454818e+05,2.776263e+05,21419.0,642077.0,18214.0,623863.0,2.235218,25340.400000,3472.268898,25310.0,30446.0,20834.0,9612.0,0.398980,100467.400000,163235.989761,27764.0,392459.0,25098.0,367361.0,2.235539,5.0,5,1,09/10/2016,29.942807,0.313382,29.9444,30.7222,29.5000,1.2222,1.002514,40.833341,0.839526,40.5556,42.2222,40.0000,2.2222,0.454498,67.808824,0.758186,68.0,69.0,66.0,3.0,-0.932719,1007.657794,0.230657,1007.720,1007.930,1007.250,0.680,-0.483309,3.526356,1.870559,3.218690,8.04672,0.00000,8.04672,0.260616
3,MA Qing Hua,Marrakesh,2016-2017,Groups Qualifying Session,Q,1.0,TECHEETAH,96.700000,21.620515,89.10,128.5,80.1,48.4,1.761566,3.075050e+04,2.285297e+04,34467.5,54067.0,0.0,54067.0,-0.879319,32754.500000,22510.967172,40873.0,49272.0,0.0,49272.0,-1.666986,17328.750000,12946.936198,19069.5,31176.0,0.0,31176.0,-0.771944,4.0,4,2,12/11/2016,19.256183,0.185109,19.2778,19.5556,18.7778,0.7778,-0.816759,22.608022,1.027137,22.7778,23.8889,20.5556,3.3333,-0.269498,65.138889,0.960737,65.0,68.0,64.0,4.0,0.936207,962.197528,0.152585,962.246,962.415,961.907,0.508,-0.495600,5.096257,2.324215,4.828030,9.65606,0.00000,9.65606,-0.247735
4,MA Qing Hua,Newyork2,2017-2018,Qualifying 2 Group 1,Q,1.0,NIO Formula E Team,74.933333,36.874698,93.10,99.2,32.5,66.7,-1.678880,5.431333e+04,3.739482e+04,33244.0,97489.0,32207.0,65282.0,1.730552,28552.333333,1275.374585,29261.0,29316.0,27080.0,2236.0,-1.728428,27890.333333,1207.169140,27685.0,29187.0,26799.0,2388.0,0.743282,3.0,3,0,07/15/2018,22.518533,0.045397,22.5278,22.5556,22.4444,0.1112,-0.857321,23.333300,0.000000,23.3333,23.3333,23.3333,0.0000,0.000000,89.333333,0.516398,89.0,90.0,89.0,1.0,0.968246,1015.210000,0.000000,1015.210,1015.210,1015.210,0.000,0.000000,3.005982,1.043103,2.652335,4.24374,2.12187,2.12187,0.455943
5,MA Qing Hua,Paris,2017-2018,Qualifying Group 4,Q,4.0,NIO Formula E Team,82.633333,33.691146,93.30,109.7,44.9,64.8,-1.281899,3.527567e+04,2.974910e+04,19071.0,69609.0,17147.0,52462.0,1.723904,27218.666667,2207.172928,27469.0,29290.0,24897.0,4393.0,-0.503816,24313.666667,3295.023874,24447.0,27540.0,20954.0,6586.0,-0.181795,3.0,3,0,04/28/2018,15.333333,0.049685,15.3333,15.3889,15.2778,0.1111,0.001509,21.666700,0.000000,21.6667,21.6667,21.6667,0.0000,0.000000,59.833333,0.408248,60.0,60.0,59.0,1.0,-2.449490,1007.153333,0.025820,1007.150,1007.180,1007.110,0.070,-0.759105,1.768222,0.866251,1.591400,3.1828

In [13]:
final_data_with_weather.to_csv('./cleaned_weather_dataset/cleaned_weather.csv')

In [15]:
final_data_with_weather.head()

,DRIVER_NAME,location,season_year,match_name,match_type,GROUP,TEAM,mean_KPH,std_KPH,med_KPH,max_KPH,min_KPH,range_KPH,skew_KPH,mean_S1,std_S1,med_S1,max_S1,min_S1,range_S1,skew_S1,mean_S2,std_S2,med_S2,max_S2,min_S2,range_S2,skew_S2,mean_S3,std_S3,med_S3,max_S3,min_S3,range_S3,skew_S3,Total_Lap_Num,PIT_TIME_cnt,cnt_CROSSING_FINISH_LINE_IN_PIT,DATE_ONLY,mean_air_temp,std_air_temp,med_air_temp,max_air_temp,min_air_temp,range_air_temp,skew_air_temp,mean_track_temp,std_track_temp,med_track_temp,max_track_temp,min_track_temp,range_track_temp,skew_track_temp,mean_HUMIDITY,std_HUMIDITY,med_HUMIDITY,max_HUMIDITY,min_HUMIDITY,range_HUMIDITY,skew_HUMIDITY,mean_PRESSURE,std_PRESSURE,med_PRESSURE,max_PRESSURE,min_PRESSURE,range_PRESSURE,skew_PRESSURE,mean_WIND_SPEED,std_WIND_SPEED,med_WIND_SPEED,max_WIND_SPEED,min_WIND_SPEED,range_WIND_SPEED,skew_WIND_SPEED
0,MA Qing Hua,Buenos,2016-2017,Groups Qualifying Session,Q,4.0,TECHEETAH,55.950000,74.034080,55.95,108.3,3.6,104.7,0.000000,1.205626e+06,1.673677e+06,1205626.5,2389095.0,22158.0,2366937.0,0.000000,28569.500000,2348.301620,28569.5,30230.0,26909.0,3321.0,0.000000,33244.500000,132.228968,33244.5,33338.0,33151.0,187.0,0.000000,2.0,2,0,02/18/2017,26.299227,0.067998,26.2778,26.4444,26.2222,0.2222,0.634238,26.250000,0.243363,26.1111,26.6667,26.1111,0.5556,1.195861,74.931818,0.899554,75.0,76.0,73.0,3.0,-0.665006,1008.236136,0.126201,1008.230,1008.470,1008.030,0.440,0.053279,4.462272,2.049350,4.828030,8.04672,0.00000,8.04672,0.095071
1,MA Qing Hua,HongKong,2016-2017,Groups Qualifying Session,Q,2.0,TECHEETAH,60.720000,44.706677,83.70,103.4,9.6,93.8,-0.504993,1.454818e+05,2.776263e+05,21419.0,642077.0,18214.0,623863.0,2.235218,25340.400000,3472.268898,25310.0,30446.0,20834.0,9612.0,0.398980,100467.400000,163235.989761,27764.0,392459.0,25098.0,367361.0,2.235539,5.0,5,1,09/10/2016,29.942807,0.313382,29.9444,30.7222,29.5000,1.2222,1.002514,40.833341,0.839526,40.5556,42.2222,40.0000,2.2222,0.454498,67.808824,0.758186,68.0,69.0,66.0,3.0,-0.932719,1007.657794,0.230657,1007.720,1007.930,1007.250,0.680,-0.483309,3.526356,1.870559,3.218690,8.04672,0.00000,8.04672,0.260616
3,MA Qing Hua,Marrakesh,2016-2017,Groups Qualifying Session,Q,1.0,TECHEETAH,96.700000,21.620515,89.10,128.5,80.1,48.4,1.761566,3.075050e+04,2.285297e+04,34467.5,54067.0,0.0,54067.0,-0.879319,32754.500000,22510.967172,40873.0,49272.0,0.0,49272.0,-1.666986,17328.750000,12946.936198,19069.5,31176.0,0.0,31176.0,-0.771944,4.0,4,2,12/11/2016,19.256183,0.185109,19.2778,19.5556,18.7778,0.7778,-0.816759,22.608022,1.027137,22.7778,23.8889,20.5556,3.3333,-0.269498,65.138889,0.960737,65.0,68.0,64.0,4.0,0.936207,962.197528,0.152585,962.246,962.415,961.907,0.508,-0.495600,5.096257,2.324215,4.828030,9.65606,0.00000,9.65606,-0.247735
4,MA Qing Hua,Newyork2,2017-2018,Qualifying 2 Group 1,Q,1.0,NIO Formula E Team,74.933333,36.874698,93.10,99.2,32.5,66.7,-1.678880,5.431333e+04,3.739482e+04,33244.0,97489.0,32207.0,65282.0,1.730552,28552.333333,1275.374585,29261.0,29316.0,27080.0,2236.0,-1.728428,27890.333333,1207.169140,27685.0,29187.0,26799.0,2388.0,0.743282,3.0,3,0,07/15/2018,22.518533,0.045397,22.5278,22.5556,22.4444,0.1112,-0.857321,23.333300,0.000000,23.3333,23.3333,23.3333,0.0000,0.000000,89.333333,0.516398,89.0,90.0,89.0,1.0,0.968246,1015.210000,0.000000,1015.210,1015.210,1015.210,0.000,0.000000,3.005982,1.043103,2.652335,4.24374,2.12187,2.12187,0.455943
5,MA Qing Hua,Paris,2017-2018,Qualifying Group 4,Q,4.0,NIO Formula E Team,82.633333,33.691146,93.30,109.7,44.9,64.8,-1.281899,3.527567e+04,2.974910e+04,19071.0,69609.0,17147.0,52462.0,1.723904,27218.666667,2207.172928,27469.0,29290.0,24897.0,4393.0,-0.503816,24313.666667,3295.023874,24447.0,27540.0,20954.0,6586.0,-0.181795,3.0,3,0,04/28/2018,15.333333,0.049685,15.3333,15.3889,15.2778,0.1111,0.001509,21.666700,0.000000,21.6667,21.6667,21.6667,0.0000,0.000000,59.833333,0.408248,60.0,60.0,59.0,1.0,-2.449490,1007.153333,0.025820,1007.150,1007.180,1007.110,0.070,-0.759105,1.768222,0.866251,1.591400,3.1828

In [16]:
from sklearn import ensemble, linear_model, neighbors, svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [17]:
from sklearn.preprocessing import OneHotEncoder

def encode_column_to_dataframe(df, allcols):
    '''Takes the column name as a string and adds to the dataframe'''
    for col in allcols:
        #dum_df = pd.get_dummies(df, columns=col, prefix=col)
        # creating instance of one-hot-encoder
        enc = OneHotEncoder(handle_unknown='ignore')
        # passing bridge-types-cat column (label encoded values of bridge_types)
        enc_df = pd.DataFrame(enc.fit_transform(df[[col]]).toarray())
        # merge with main df bridge_df on key values
        df = df.join(enc_df.add_prefix(col+"_"))
    return df

In [18]:
def get_encoded_dataframe(final):
    final_data = encode_column_to_dataframe(final,['DRIVER_NAME','location','season_year', 'match_name','match_type','GROUP', 'TEAM'])
    final_data.drop(columns = ['DRIVER_NAME','location','season_year', 'match_name','match_type','GROUP', 'TEAM'],inplace = True)
    #final_copy.to_csv('./cleaned/final_processed_before_ml.csv')
    #drop only if that column exists originally
    #final_data.drop(columns = ['Unnamed: 0'],inplace = True)
    return final_data

In [20]:
def test_model(X, y, model_name, model):
    print("MODEL: {}".format(model_name))
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    model.fit(X_train, y_train)
    Z = model.predict(X_test)
    print("R2 Value: ", r2_score(Z, y_test))
    print("MSE Value: ", mean_squared_error(Z, y_test))
    
models = {
    #"RANSACRegressor": linear_model.RANSACRegressor(random_state=0),
    "ElasticNet": linear_model.ElasticNet(random_state=0),
    "Random Forest": ensemble.RandomForestRegressor(n_estimators=1000),
    "K-Nearest Neighbors": neighbors.KNeighborsRegressor()
    
}

In [19]:
final_df = get_encoded_dataframe(final_data_with_weather)
final_df = final_df.fillna(0)
y = final_df['Total_Lap_Num'].to_numpy()
X = final_df.drop(columns = ['Total_Lap_Num','DATE_ONLY']).to_numpy()

In [21]:
for model in models.keys():
    print('----------------------\n')
    test_model(X,y,model, models[model])
    

----------------------

MODEL: ElasticNet


C:\Users\Aishwarya\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 348.45044964041324, tolerance: 2.1780580037664783
  positive)


R2 Value:  0.9964246204079967
MSE Value:  0.13328850950937504
----------------------

MODEL: Random Forest
R2 Value:  0.9996748784552001
MSE Value:  0.009272736842105249
----------------------

MODEL: K-Nearest Neighbors
R2 Value:  0.7430903828033678
MSE Value:  12.640601503759399
